In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

print(__doc__)

# #############################################################################
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.
tmin, tmax = -1., 4.
event_id = dict(hands=2, feet=3)
subject = 1
runs = [6, 10, 14]  # motor imagery: hands vs feet

raw_fnames = eegbci.load_data(subject, runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])

# strip channel names of "." characters
raw.rename_channels(lambda x: x.strip('.'))

# Apply band-pass filter
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
labels = epochs.events[:, -1] - 2

Automatically created module for IPython interactive environment
Extracting EDF parameters from C:\Users\student\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\student\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S001\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\student\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass

In [5]:
y_train.shape

(36,)

In [3]:
csp = CSP(n_components=64, reg=None, log=None, norm_trace=False,transform_into='csp_space')

epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)
for train_idx, test_idx in cv_split:
    print(train_idx,test_idx)
    y_train, y_test = labels[train_idx], labels[test_idx]

    X_train = csp.fit_transform(epochs_data_train[train_idx], y_train)
    X_test = csp.transform(epochs_data_train[test_idx])

[ 3  6 24 32 19 17 13 42 15  9 16 37 31 27  0 30 29  5 11 33  1 40 21  2
 34 23 36 10 22 18 44 20  7 14 28 38] [39 25 26 43 35 41  4 12  8]
Computing data rank from raw with rank=None
    Using tolerance 9.7e-05 (2.2e-16 eps * 64 dim * 6.8e+09  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing data rank from raw with rank=None
    Using tolerance 0.0001 (2.2e-16 eps * 64 dim * 7.3e+09  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
[32 41 20 36 29 15 14 33 23 43 37 11 30 40 28  9 21  5 39 31 35 17 44 16
 42 34  7  6 27 19  1 25  8 13 24  3] [ 2  0 26 22 12 38 10 18  4]
Computing data rank from raw with rank=None
    Using tolerance 9.5e-05 (2.2e-16 eps * 64 dim * 6.7e+09  max singular value)
  

In [17]:
labels

array([1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1])

In [4]:
print(epochs_data_train[train_idx].shape)

(36, 64, 161)


In [11]:
epochs_data_train

array([[[ 7.41003696e-08,  7.67937359e-06,  8.28728506e-06, ...,
         -3.59676869e-06, -1.70368554e-05, -2.20881031e-05],
        [-1.45987909e-06,  6.02861535e-06,  6.90751562e-06, ...,
         -1.42806042e-05, -2.46842275e-05, -2.46151731e-05],
        [-1.16161688e-05, -1.22545977e-06,  5.11143148e-06, ...,
         -1.29178509e-05, -1.83998041e-05, -1.92278408e-05],
        ...,
        [ 4.48779566e-06,  5.37720556e-06,  8.85273511e-06, ...,
          3.94623804e-05,  3.74264935e-05,  1.75330420e-05],
        [ 2.44260252e-05,  2.95298648e-05,  3.12884873e-05, ...,
          4.33848498e-05,  2.98571620e-05,  6.21245592e-06],
        [ 1.19242062e-05,  1.48466229e-05,  1.39721839e-05, ...,
          3.05620585e-05,  3.40502733e-05,  2.02348830e-05]],

       [[-6.27576188e-06, -5.82008405e-07,  3.63030896e-06, ...,
         -1.66813520e-05, -1.41266917e-05, -1.17255292e-05],
        [-3.42076611e-06, -1.68350965e-06, -1.67066580e-06, ...,
         -1.58844609e-05, -1.96338345e

In [13]:
epochs_data_train[1].shape

(64, 161)

In [5]:
set(labels)

{0, 1}

In [6]:
epochs_data.shape

(45, 64, 801)

In [5]:
epochs_data_train[train_idx].shape,y_train.shape,epochs_data_train[test_idx].shape,y_test.shape

((36, 64, 161), (36,), (9, 64, 161), (9,))

In [15]:
# Define a monte-carlo cross-validation generator (reduce variance):

# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('CSP', csp), ('LDA', lda)])#
clf.fit(epochs_data_train[train_idx],y_train)

Computing data rank from raw with rank=None
    Using tolerance 0.00011 (2.2e-16 eps * 64 dim * 7.7e+09  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing data rank from raw with rank=None
    Using tolerance 0.0001 (2.2e-16 eps * 64 dim * 7.3e+09  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.


Pipeline(memory=None,
     steps=[('CSP', CSP(cov_est='concat', cov_method_params=None, log=True, n_components=4,
  norm_trace=False, rank=None, reg=None, transform_into='average_power')), ('LDA', LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001))])

In [19]:
y_pred=clf.predict(epochs_data_train[test_idx])
y_pred

array([1, 0, 1, 0, 0, 1, 0, 1, 1])

In [20]:
y_true=y_test
y_true

array([1, 0, 1, 0, 0, 0, 0, 1, 1])

In [21]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print('accuracy:',accuracy_score(y_true,y_pred))
print(classification_report(y_true,y_pred))

accuracy: 0.8888888888888888
             precision    recall  f1-score   support

          0       1.00      0.80      0.89         5
          1       0.80      1.00      0.89         4

avg / total       0.91      0.89      0.89         9

